## Text pre-processing

In [1]:
# Loading libraries
import numpy as np
import pandas as pd
import requests
import io
from io import BytesIO
from google.colab import drive
drive.mount('/content/drive')
from google.colab import files

Mounted at /content/drive


In [5]:
path = '/content/drive/MyDrive/Preprocessed_tweets_data_twint.csv'
df = pd.read_csv(path)
df.head()

,Unnamed: 0,Unnamed: 0.1,date/time range,timezone,user_id,username,place,tweet,hashtags,Cleaned Tweets,category
0,0,0,2020-10-31 07:56:37,800,17580230,jarallen,Not Specified,@austinperroux @NWSSanAntonio Same here. I’ve ...,NoTags,"['austinperroux', 'NWSSanAntonio', 'I’ve', 'ne...",flood
1,1,1,2020-10-31 07:49:35,800,472122299,philippinestar,Not Specified,"Flooding (including flash floods), rain-induce...",NoTags,"['Flooding', 'including', 'flash', 'floods', '...",flood
2,2,2,2020-10-31 07:23:57,800,2453732882,ttweathercenter,Not Specified,7:20 PM - Flash flooding continues in Arouca t...,NoTags,"['720', 'PM', 'Flash', 'flooding', 'continues'...",flood
3,3,3,2020-10-31 07:12:06,800,2453732882,ttweathercenter,Not Specified,Another video of the flash flooding ongoing at...,NoTags,"['Another', 'video', 'flash', 'flooding', 'ong...",flood
4,4,4,2020-10-31 07:10:16,800,2453732882,ttweathercenter,Not Specified,7:05 PM - Flash flooding ongoing at Henry Stre...,NoTags,"['705', 'PM', 'Flash', 'flooding', 'ongoing', ...",flood


In [6]:
#Code to display all details in the columns
pd.set_option('display.max_columns', None)  
pd.set_option('display.expand_frame_repr', False)
pd.set_option('max_colwidth', -1)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:4: FutureWarning: Passing a negative integer is deprecated in version 1.0 and will not be supported in future version. Instead, use None to not limit the column width.
  after removing the cwd from sys.path.


In [7]:
# Converting emojis and emoticons to words
!pip install emot
from emot.emo_unicode import UNICODE_EMO, EMOTICONS
# Converting emojis to words
import re
def convert_emojis(text):
    for emot in UNICODE_EMO:
        text = text.replace(emot, "_".join(UNICODE_EMO[emot].replace(",","").replace(":","").split()))
        return text
# Converting emoticons to words    
def convert_emoticons(text):
    for emot in EMOTICONS:
        text = re.sub(u'('+emot+')', "_".join(EMOTICONS[emot].replace(",","").split()), text)
        return text
# Passing both functions to 'text_rare'
df['tweet'] = df['tweet'].apply(convert_emoticons)
df['tweet'] = df['tweet'].apply(convert_emojis)

In [8]:
import nltk
from nltk.tokenize import RegexpTokenizer # For tokenization
from nltk.stem import WordNetLemmatizer,PorterStemmer # For lemmatization
from nltk.corpus import stopwords# To remove stop words
nltk.download('stopwords')
nltk.download('wordnet')
import re
lemmatizer = WordNetLemmatizer()
stemmer = PorterStemmer() 

def preprocess(sentence):
    sentence=str(sentence)
    sentence = sentence.lower()
    sentence=sentence.replace('{html}',"") # removing html files
    cleanr = re.compile('<.*?>') # Removing punctuation
    cleantext = re.sub(cleanr, '', sentence)
    rem_url=re.sub(r'http\S+', '',cleantext) # Removing links
    rem_num = re.sub('[0-9]+', '', rem_url)  # Removing numbers
    tokenizer = RegexpTokenizer(r'\w+') # Tokenization
    tokens = tokenizer.tokenize(rem_num)  
    filtered_words = [w for w in tokens if len(w) > 2 if not w in stopwords.words('english')]
    stem_words=[stemmer.stem(w) for w in filtered_words]
    lemma_words=[lemmatizer.lemmatize(w) for w in stem_words]
    return " ".join(filtered_words)

# Mapping our loop to our datasets
df['tweet']=df['tweet'].map(lambda s:preprocess(s))
df['tweet']=df['tweet'].map(lambda s:preprocess(s))

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


In [9]:
# Spelling correction
!pip install textblob
# Spell check using text blob for the first 5 records
from textblob import TextBlob
df['tweet'][:5].apply(lambda x: str(TextBlob(x).correct()))

0    austinperroux nwssanantonio never seen water rise glad river creeks flash flood extent ever present danger along corridor never forget events                                
1    flooding including flash floods rain induced landslide sediment laden streamflows later may occur heavy prolonged rainfall especially areas highly highly susceptible hazards
2    flash flooding continues around tonight time avoid entering flood waters                                                                                                     
3    another video flash flooding going henry street around time thunderstorm affects area avoid entering flood waters                                                            
4    flash flooding going henry street around time avoid entering flood waters                                                                                                    
Name: tweet, dtype: object

In [10]:
# Common word removal
# Checking the first 10 most frequent words
from collections import Counter
cnt = Counter()
for text in df["tweet"].values:
    for word in text.split():
        cnt[word] += 1
        
cnt.most_common(10)

[('flood', 13148),
 ('flash', 3722),
 ('floods', 3664),
 ('flooding', 3620),
 ('flooded', 3248),
 ('amp', 3117),
 ('inundating', 3096),
 ('inundation', 3048),
 ('inundated', 3012),
 ('disaster', 2461)]